In [1]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import pandas as pd
from astropy import units as u
import sklearn.cluster as sclust
import sklearn.metrics as smet
import scipy.optimize as optimize
from astropy.coordinates import SkyCoord
import seaborn
from scipy.integrate import quad
from scipy.integrate import romberg 
from iminuit import Minuit
from iminuit import minimize
from SN_distribution import SkyMap
from SN_distribution import Patching_Map
from Selections import Criteria_selection
from Selections import Good_sampling
from Selections import salt_2_selection
from Hubble import Hubble_fit
from Hubble import Hubble_fit_patch
from applySelection import select_end
from applyFit import FIT
import csv
import time

ModuleNotFoundError: No module named 'healpy'

In [2]:
import warnings

# Filtrer les avertissements spécifiques
warnings.filterwarnings("ignore", message="A value is trying to be set on a copy of a slice from a DataFrame")


In [4]:
#with open('Sensi.csv', 'w') as f:
#        writer = csv.writer(f)
#        headers = ["Sim", "Method", "Mean", "std"]
 #       writer.writerow(headers)
for i in range(0, 10):
    start = time.time()
    df_SNeIa = pd.read_pickle('input/sim3000{}/salt2_fit_lc.pkl'.format(i))
    selct = select_end(df_SNeIa)
    print('Selection on the simulation n°{}'.format(i))
    selct.allSelct(n = i, method = 'fixe', N_patch = 12, N_resol = 1, H0 = 70)
    df = pd.read_pickle('output/selection_data_3years(3000{})_fix.pkl'.format(i))
    fit = FIT(df)
    delta_H0 = (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
    print('fit on simulation n°{}'.format(i))
    fit.analysis(H0 = 70, delta_H0 = delta_H0, method = 'fixe', 
             n = i, n_dip = 1, N_patch = 12, N_resol = 1, anisotropies = False, 
             theta_dip = 0, phi_dip = 0, delta_H0_dip = 5, H0_dip = 70, filename = 'output/Sensi.csv', 
                 filename_residu = 'output/res.csv')
    stop = time.time()
    print('time=', (stop - start)/3600, "hr")


Selection on the simulation n°0


/Users/antoine/Documents/newCode/SN_distribution.py:440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ncluster'] = ipix


fit on simulation n°0
time= 0.128981039457851 hr
Selection on the simulation n°1


KeyboardInterrupt: 

In [ ]:
    df = pd.read_pickle('output/selection_data_3years(3000{})_fix.pkl'.format(0))
df

In [ ]:
# Ecliptic
plan_lon = np.linspace(0, 360, 10000)
plan_lat = np.zeros(10000)
ecl = SkyCoord(plan_lon, plan_lat, unit = 'deg', frame = 'geocentrictrueecliptic') # use astropy for plan coordinates
ra_ecl, dec_ecl = ecl.icrs.ra.value, ecl.icrs.dec.value # Transforms ecliptic coordinates into equatorial
            # Transforms degree into radian
theta_ecl = np.radians(90. - dec_ecl)
phi_ecl = np.radians(ra_ecl)
            # Galactic plan
gal = SkyCoord(plan_lon, plan_lat, frame = 'galactic', unit = 'degree') # use astropy for plan coordinates
ra_gal, dec_gal = gal.icrs.ra.value, gal.icrs.dec.value # Transforms galactic coordinates into equatorial
            # Transforms degree into radian
theta_gal = np.radians(90. - dec_gal)
phi_gal = np.radians(ra_gal)
            # Mollweide view
hp.mollview(title = 'Equatorial coordinates representation of {}'.format('pion'))
hp.graticule()
psc = hp.projscatter(np.pi/2 - np.radians(df['dec']), np.radians(df['ra']), c = df['ncluster'], s = 3)
plt.colorbar(psc, label = 'N_cluster')
            # Projection of the different plans
hp.projplot(theta_gal, phi_gal, color = 'red', label = 'Galactic')
hp.projplot(theta_ecl, phi_ecl, color = 'blue', label = 'Ecliptic')
plt.legend()